<a href="https://colab.research.google.com/github/sblHead/Supplementary-code/blob/main/INN_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from google.colab import drive
import csv
import sklearn


drive.mount('/content/drive/')    # if using google drive

Mounted at /content/drive/


In [ ]:
input_shape = 81
Xdata = []

with open('', newline = '\n') as csvfile:     # read in material data
  data = csv.reader(csvfile, delimiter = ',')
  for row in data:
    row = np.array(row)
    row =  row.astype(np.float)
    Xdata.append(row)

Xdata = np.delete(Xdata,slice(1,len(Xdata),2),axis = 0)

Ydata = []
with open('', newline = '\n') as csvfile:   # read in spectral response data
  data = csv.reader(csvfile, delimiter = ',')
  for row in data:
    row = np.array(row)
    row =  row.astype(np.float)
    Ydata.append(row)

Ydata = np.transpose(Ydata)
Ydata = np.delete(Ydata,slice(1,(len(Ydata)),2),axis = 0)


print(np.shape(Xdata))   # ensure that each row is a new sample, i.e. shape = (num samples, len sample)
print(np.shape(Ydata))


In [ ]:
SNN_model1 = keras.models.load_model('')      # load SNN model from saved location
SNN_model = keras.models.clone_model(SNN_model1)
SNN_model.set_weights(SNN_model1.get_weights())
for layer in SNN_model.layers:
  layer.trainable = False

SNN_model.summary()

In [ ]:
epoch_num = 40


XTrain = []
YTrain = []
XTest = []
YTest = []
XNew = []
YNew = []

XShuff, YShuff = sklearn.utils.shuffle(Ydata, Yspectrum)

XTrain = XShuff[:18000]
YTrain = YShuff[:18000]
XTest = XShuff[18000:21000]
YTest = YShuff[18000:21000]
XNew = XShuff[21000:]
YNew = YShuff[21000:]

XTrain = np.array(XTrain)
YTrain = np.array(YTrain)
XTest = np.array(XTest)
YTest = np.array(YTest)


In [ ]:

input_tensor = keras.Input(shape = (input_shape))
input_tensor= keras.layers.BatchNormalization(axis=-1 , center=True , scale=True)(input_tensor)

x = keras.layers.Dense(1000, activation = None)(input_tensor)  #input to layer 1
x = keras.layers.LeakyReLU(alpha = 0.2)(x)


x = keras.layers.Dense(1000, activation = None)(x)    #layer 2
x = keras.layers.LeakyReLU(alpha = 0.2)(x)


x = keras.layers.Dense(1000, activation = None)(x)    #layer 3
x = keras.layers.LeakyReLU(alpha = 0.2)(x)
x = keras.layers.Dropout(0.02)(x)

x = keras.layers.Dense(500, activation = None)(x)   #layer 4
x = keras.layers.LeakyReLU(alpha = 0.2)(x)
x = keras.layers.Dropout(0.01)(x)

x = keras.layers.Dense(500, activation = None)(x)   #layer 5
x = keras.layers.LeakyReLU(alpha = 0.2)(x)

x = keras.layers.Dense(500, activation = None)(x)   #layer 6
x = keras.layers.LeakyReLU(alpha = 0.2)(x)
output_tensor = keras.layers.Dense(6, activation = 'sigmoid')(x)   #output

inverse_model = keras.models.Model(input_tensor, output_tensor)

tandem_model_input = keras.Input(shape=(input_shape, ))
intermediate = inverse_model(tandem_model_input)
tandem_output = SNN_model(intermediate)
tandem_model = keras.models.Model(tandem_model_input, tandem_output)


tandem_model.compile(loss = keras.losses.MeanSquaredError(),
              optimizer = 'Adam',
              metrics = ['mean_absolute_error'])


callback_list = [keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=4, min_lr=1e-7), 
                 keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)]


history = tandem_model.fit(XTrain, YTrain, epochs = epoch_num, callbacks = callback_list, validation_data = (XTest,  YTest))
print(tandem_model.evaluate(YNew, YNew, verbose = 0))

#tandem_model.summary()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
loss_train = history.history['mean_absolute_error']     # learning curve plot (validation and training curves)
loss_val = history.history['val_mean_absolute_error']
epochs = range(1,22)
plt.figure(figsize = (3.5,3.5),dpi=1200)
plt.plot(epochs, loss_train[1:], 'g', label='Training MSE')
plt.plot(epochs, loss_val[1:], 'b', label='Validation MSE')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Mean Absolute Error',fontsize=16)
plt.show()

In [ ]:
num = ##                       # plots target vs predicted spectra
Y_hat = tandem_model.predict(Y_arb[num].reshape(-1,81))
Y_acc1 = Y_arb[num].reshape(81, -1)
Y_hat = np.transpose(Y_hat)
spectrum_wav = np.arange(380, 781, 5)
# plt.figure(figsize = (3.5,3.5),dpi=1200)
plt.plot(spectrum_wav, Y_hat, 'b')
plt.plot(spectrum_wav, Y_acc1, 'r')
plt.ylim([0, 1])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Wavelength, nm', fontsize=16)
plt.ylabel('Reflectance coefficient',fontsize=16)
plt.show()